In [1]:
!pip install torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl
!pip install torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
!pip install torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Processing ./torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python3.7 -m pip install --upgrade pip' command.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Processing ./torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python3.7 -m pip install --upgrade pip' command.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Processing ./torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
!pip install torch-cluster===1.5.9 torch-geometric===2.0.2 torch-scatter===2.0.9 torch-sparse===0.6.12 pyamg

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 325 kB 34.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 528 kB 30.4 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 16.2 MB/s eta 0:00:01
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535547 sha256=9b9c9da8838634468df77ae90216bc44142f322a71e38c732d47efec18a7e02a
  Stored in directory: /home/ma-user/.cache/pip/wheels/6d/f6/f7/176a3c99c3b84c4d5890bf983263779376c8cad1028d8f4d04
Successfully built torch-geometric
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [3]:
import os
import sys
import math
import functools
import collections
from collections import OrderedDict
import numpy as np
import scipy.sparse as sp
import scipy.spatial.qhull
import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torch_geometric
import torch_scatter
import torch_sparse
import pyamg
   
from torch_geometric.utils import (
   to_scipy_sparse_matrix,
   add_remaining_self_loops,
   remove_self_loops,
   to_dense_adj,
   subgraph,
   from_scipy_sparse_matrix
)
from torch_geometric.nn import knn_interpolate
from torch_geometric.data import Data, Batch, Dataset
from torch_geometric.loader import DataLoader
   
from pyamg.classical.split import RS, CLJP

from torch_scatter.composite import scatter_softmax

from torch_sparse import SparseTensor, coalesce, transpose, spspmm

import pickle
from pathlib import Path
import math,copy
from os import PathLike
from typing import Sequence, Dict, Union, Tuple, List
from torch._six import string_classes
import collections.abc as container_abcs
from scipy.spatial.qhull import Delaunay
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision import utils
from torchvision.utils import make_grid

device=torch.device("cuda:0")

/home/ma-user/anaconda3/envs/PyTorch-1.10.2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# AMGNet模型

In [4]:
def getcorsenode(latent_graph):
    A=to_scipy_sparse_matrix(latent_graph.edge_index).tocsr()
    splitting=RS(A)
    index=np.array(np.nonzero(splitting))
    b = torch.from_numpy(index)
    b=torch.squeeze(b)
    return b

getcorsenode函数将输入图转换为稀疏矩阵，然后应用 Ruge-Stubbe 分裂算法来识别图中的“粗”节点，最后将这些节点的索引以 PyTorch 张量的形式返回。

In [5]:
def StAS(index_A, value_A, index_S, value_S,N, kN,nor):

    index_A, value_A = coalesce(index_A, value_A, m=N, n=N)
    index_S, value_S = coalesce(index_S, value_S, m=N, n=kN)
    index_B, value_B = spspmm(index_A, value_A, index_S, value_S, N, N, kN)
    index_St, value_St = transpose(index_S, value_S, N, kN)
    index_B, value_B = coalesce(index_B, value_B, m=N, n=kN)                             
    index_E, value_E = spspmm(index_St, value_St, index_B, value_B, kN, N, kN)

    return index_E, value_E

StAS函数计算了稀疏矩阵 $S^T * A * S$。它首先规范化输入矩阵，然后分两步进行稀疏矩阵乘法：先计算 $A * S$，再计算 $S^T * (A * S)$。中间结果和最终结果都保持了稀疏格式。

In [6]:
def graph_connectivity(perm, edge_index, edge_weight, score, pos, N, nor):
    
    # 获取被选中的节点数量
    kN = perm.size(0)
    perm2 = perm.view(-1, 1)
    
    # 创建一个布尔掩码，标识哪些边起源于 perm (选中的) 节点
    mask = (edge_index[0] == perm2).sum(0, dtype=torch.bool)
    
    # 创建 S 矩阵
    # S0: 选中的节点连接的邻居节点索引
    S0 = edge_index[1][mask].view(1, -1)
    # S1: 选中的节点索引
    S1 = edge_index[0][mask].view(1, -1)
    index_S = torch.cat([S0, S1], dim=0)
    value_S = score[mask].detach().squeeze()
    
    # 重新标记用于池化的索引，使 S 的形状为 [N x kN]
    n_idx = torch.zeros(N, dtype=torch.long)
    # 将 perm 索引映射到新的索引范围
    n_idx[perm] = torch.arange(perm.size(0))
    index_S[1] = n_idx[index_S[1]]

    # 获取子图节点的位置信息
    subgraphnode_pos = pos[perm].cpu()
    subgraphnode_pos = subgraphnode_pos.to(device)
    
    # 创建 A 矩阵
    # 复制原始的边索引
    index_A = edge_index.clone()
    # 生成边权重 value_A
    if edge_weight is None:
        value_A = value_S.new_ones(edge_index[0].size(0))
    else:
        value_A = edge_weight.clone()
    value_A = torch.squeeze(value_A)

    # 初始化属性列表
    attrlist = []

    for i in range(128):
        # 调用 StAS 函数，计算新的边索引和属性
        index_E, value_E = StAS(index_A, value_A[:, i], index_S, value_S, N, kN, nor)
        index_E, value_E = remove_self_loops(edge_index=index_E, edge_attr=value_E)
        attrlist.append(value_E)
    # 将属性列表堆叠成一个张量
    edge_weight = torch.stack(attrlist, dim=1)
    
    return index_E, edge_weight, subgraphnode_pos

graph_connectivity 函数通过利用原始图的连接结构、边权重和边的贡献分数，使用稀疏矩阵乘法 (StAS) 将原始图的连接性信息投影到池化后的子图上，从而为子图生成新的、信息丰富的边权重。这个过程是自适应结构感知池化 (ASAP) 方法的一部分，旨在学习层次化的图表示。
1. **识别相关边:** 首先，函数识别出所有起点（或终点，取决于 `mask` 的具体含义）位于被选中节点（`perm`）的边。
2. **构建投影矩阵 S:** 使用这些相关边的终点（或起点）作为索引，并使用这些边对应的 `score` 作为权重，构建一个稀疏矩阵 S。这个矩阵 S 代表了从原始图到子图的投影关系。为了适应池化操作，对 S 的索引进行了重新标记。
3. **准备原始图的邻接矩阵 A:** 使用原始图的边索引，并根据是否提供了 `edge_weight` 来决定邻接矩阵 A 的权重（如果未提供，则使用全1权重）。
4. **计算子图连接性:** 这是核心步骤。函数通过多次调用 `StAS` 函数来计算子图的连接性权重。`StAS` 计算的是 `S^T A S`（S的转置乘以A再乘以S）。这个操作将原始图的连接性信息（通过 A 表示）通过投影矩阵 S 映射到子图空间。`StAS` 被调用多次（示例中是128次），可能是为了处理不同类型的特征或属性。
5. **后处理:** 对 `StAS` 的结果进行后处理，例如移除自环（self-loops）。
6. **输出:** 最终，函数返回子图的边索引 (`index_E`)、计算得到的新边权重 (`edge_weight`) 以及子图中节点的位置信息 (`subgraphnode_pos`)。

In [7]:
class GraphNetBlock(nn.Module):
    """具有残差连接的多边缘交互网络"""

    def __init__(self, model_fn, output_size, message_passing_aggregator, attention=False):
        super().__init__()
        self.edge_model = model_fn(output_size)
        self.node_model = model_fn(output_size)
        self.message_passing_aggregator = message_passing_aggregator

    def _update_edge_features(self, graph):
        """聚合节点特征，并应用边缘功能"""
        senders = graph.edge_index[0].to(device)
        receivers = graph.edge_index[1].to(device)
        sender_features = torch.index_select(input=graph.x, dim=0, index=senders)
        receiver_features = torch.index_select(input=graph.x, dim=0, index=receivers)
        features = [sender_features, receiver_features, graph.edge_attr]
        features = torch.cat(features, dim=-1)
        return self.edge_model(features)

    def unsorted_segment_operation(self, data, segment_ids, num_segments, operation):
        """
        计算张量线段的和。类似于 tf.unsorted_segment_sum。

        :param data： 张量，需要对其进行分段求和。
        :param segment_ids： 分段索引张量。
        :param num_segments： 线段的数量。
        :return： 与数据参数类型相同的张量。
        """
        assert all([i in data.shape for i in segment_ids.shape]), "segment_ids.shape should be a prefix of data.shape"

        # segment_ids is a 1-D tensor repeat it to have the same shape as data
        if len(segment_ids.shape) == 1:
            s = torch.prod(torch.tensor(data.shape[1:])).long().to(device)
            segment_ids = segment_ids.repeat_interleave(s).view(segment_ids.shape[0], *data.shape[1:]).to(device)

        assert data.shape == segment_ids.shape, "data.shape and segment_ids.shape should be equal"

        shape = [num_segments] + list(data.shape[1:])
        result = torch.zeros(*shape)
        if operation == 'sum':
            result = torch_scatter.scatter_add(data.float(), segment_ids, dim=0, dim_size=num_segments)
        elif operation == 'max':
            result, _ = torch_scatter.scatter_max(data.float(), segment_ids, dim=0, dim_size=num_segments)
        elif operation == 'mean':
            result = torch_scatter.scatter_mean(data.float(), segment_ids, dim=0, dim_size=num_segments)
        elif operation == 'min':
            result, _ = torch_scatter.scatter_min(data.float(), segment_ids, dim=0, dim_size=num_segments)
        result = result.type(data.dtype)
        return result

    def _update_node_features(self, node_features, edge_attr, edge_index):
        """聚合边缘特征，并应用节点功能"""
        num_nodes = node_features.shape[0]
        features = [node_features]
        features.append(
            self.unsorted_segment_operation(edge_attr, edge_index[1], num_nodes,
                                            operation=self.message_passing_aggregator))
        features = torch.cat(features, dim=-1)
        return self.node_model(features)

    def forward(self, graph):
        """应用 GraphNetBlock 并返回更新后的 MultiGraph"""
        new_edge_features = self._update_edge_features(graph)
        new_node_features = self._update_node_features(graph.x, graph.edge_attr, graph.edge_index)

        new_node_features += graph.x
        new_edge_features += graph.edge_attr
        return Data(x=new_node_features, pos=graph.pos, edge_attr=graph.edge_attr,
                    edge_index=graph.edge_index, batch=graph.batch)

GraphNetBlock 是一个典型的图神经网络块，它通过边更新和节点更新两个步骤来传递和聚合信息，并使用残差连接来增强学习能力和稳定性。它允许网络在每一层都学习节点和边之间的复杂交互关系，是构建更深层次图神经网络的基础单元。
### 模块结构
1.  **初始化 (`__init__`)**:
    *   接收参数：`model_fn`（一个用于创建子模型的函数，通常是 MLP）、`output_size`（输出特征维度）、`message_passing_aggregator`（消息聚合方式，如 'sum', 'mean', 'max' 等）、`attention`（是否使用注意力机制，代码中相关部分被注释掉了）。
    *   成员变量：
        *   `edge_model`: 通过 `model_fn` 创建，用于更新边的特征。
        *   `node_model`: 通过 `model_fn` 创建，用于更新节点的特征。
        *   `message_passing_aggregator`: 存储消息聚合方式。
        *   `attention_model`: (注释掉) 如果使用注意力机制，会创建一个注意力模型。
2.  **内部方法**:
    *   `_update_edge_features(graph)`: 负责更新图中的边特征。
    *   `unsorted_segment_operation(data, segment_ids, num_segments, operation)`: 一个辅助函数，实现类似 TensorFlow 中 `unsorted_segment_sum` 的功能，用于对张量进行分段求和、求最大值、求平均值等操作。它使用了 `torch_scatter` 库。
    *   `_update_node_features(node_features, edge_attr, edge_index)`: 负责更新图中的节点特征。
3.  **前向传播 (`forward`)**: 定义了模块接收输入（图数据）并产生输出（更新后的图数据）的计算过程。
### 功能
`GraphNetBlock` 的核心功能是执行图神经网络中的一个基本计算步骤，它包含两个主要的更新过程和一个残差连接：
1.  **边特征更新 (`_update_edge_features`)**:
    *   对于图中的每条边，它获取该边的两个端点（发送节点和接收节点）的特征，以及该边自身的特征。
    *   将这些特征拼接在一起，形成一个组合特征向量。
    *   将这个组合特征向量输入到 `edge_model` 中，得到更新后的边特征。
2.  **节点特征更新 (`_update_node_features`)**:
    *   对于图中的每个节点，它首先保留节点自身的原始特征。
    *   然后，它收集与该节点相连的所有边的更新后特征（通过 `unsorted_segment_operation` 函数，根据边的接收节点索引进行聚合，聚合方式由 `message_passing_aggregator` 决定）。
    *   将节点自身的特征和聚合后的边特征拼接在一起。
    *   将这个组合特征向量输入到 `node_model` 中，得到更新后的节点特征。
3.  **残差连接 (Residual Connection)**:
    *   在得到新的节点特征和边特征后，将它们与输入图中的原始节点特征和原始边特征相加。这有助于稳定训练过程，并允许网络学习恒等映射。
4.  **输出**:
    *   模块最终返回一个新的图数据对象 `Data`，其中包含了更新后的节点特征 (`new_node_features`)、原始的位置信息 (`graph.pos`)、更新后的边特征 (`new_edge_features`)、原始的边索引 (`graph.edge_index`) 和原始的批处理信息 (`graph.batch`)。

In [8]:
min_nodes = 2000  # 每个网格的粗化点都少于此值

class Processor(nn.Module):

    def __init__(self, make_mlp, output_size, message_passing_steps, message_passing_aggregator, attention=False,
                 stochastic_message_passing_used=False):
        super().__init__()
        self.stochastic_message_passing_used = stochastic_message_passing_used
        self.graphnet_blocks = nn.ModuleList()
        self.cofe_edge_blocks = nn.ModuleList()
        self.pool_blocks = nn.ModuleList()
        self.latent_size = output_size
        self.normalization = nn.LayerNorm(128)
        for index in range(message_passing_steps):
            self.graphnet_blocks.append(GraphNetBlock(model_fn=make_mlp, output_size=output_size,
                                                      message_passing_aggregator=message_passing_aggregator,
                                                      attention=attention))

            self.pool_blocks.append(GraphNetBlock(model_fn=make_mlp, output_size=output_size,
                                                  message_passing_aggregator=message_passing_aggregator,
                                                  attention=attention))
    def forward(self, latent_graph, normalized_adj_mat=None):
        x = []
        pos = []
        new = []
        for (graphnet_block, pool) in zip(self.graphnet_blocks, self.pool_blocks):
            if latent_graph.x.shape[0] > min_nodes:
                pre_matrix = graphnet_block(latent_graph)
                x.append(pre_matrix)
                cofe_graph = pool(pre_matrix)  # updata edge features
                coarsenodes = getcorsenode(pre_matrix).to(device)
                nodesfeatures = cofe_graph.x[coarsenodes]
                subedge_index, edge_weight, subpos = graph_connectivity(perm=coarsenodes,
                                                                        edge_index=cofe_graph.edge_index,
                                                                        edge_weight=cofe_graph.edge_attr,
                                                                        score=cofe_graph.edge_attr[:, 0],
                                                                        pos=cofe_graph.pos,
                                                                        N=cofe_graph.x.size(0),
                                                                        nor=self.normalization)
                edge_weight = self.normalization(edge_weight)
                pos.append(subpos)
                latent_graph = Data(x=nodesfeatures, pos=subpos,
                                    edge_attr=edge_weight, edge_index=subedge_index)
            else:
                latent_graph = graphnet_block(latent_graph)
                new.append(latent_graph)
        if len(new):
            x.append(new[-1])
        return x, pos

###  `Processor` 层
`Processor` 是一个继承自 `nn.Module` 的 PyTorch 模块，用于处理图数据。它的核心思想是：
1.  **选择关键节点:** 根据节点的特征重要性（这里使用特征平方和）选择一部分节点作为“高影响力节点”。
2.  **建立连接:** 在每个“涟漪”（ripple）层中，将选定的节点与这些高影响力节点建立连接。这些连接（包括特征和距离信息）是可学习的。
3.  **特征处理:** 使用一个多层感知器（MLP）来处理这些连接信息。
4.  **整合与输出:** 将处理后的结果添加到编码器的输出（潜在图）中，然后这个修改后的图被送回原始处理器（或下一个处理阶段）。
这个 `Processor` 类是一个 PyTorch 模块，用于处理图数据，特别是进行图神经网络中的消息传递和池化操作。它通过一系列的图神经网络块和池化块来逐步处理图数据，并在每一步中根据图的大小决定是否进行池化操作。

### 类结构和功能

1. **初始化 (`__init__` 方法)**:
   - **参数**:
     - `make_mlp`: 一个函数，用于创建多层感知机（MLP）模型。
     - `output_size`: 输出特征的维度。
     - `message_passing_steps`: 消息传递的步数。
     - `message_passing_aggregator`: 消息传递的聚合方式。
     - `attention`: 是否使用注意力机制。
     - `stochastic_message_passing_used`: 是否使用随机消息传递。
   - **成员变量**:
     - `stochastic_message_passing_used`: 是否使用随机消息传递。
     - `graphnet_blocks`: 存储图神经网络块的列表。
     - `cofe_edge_blocks`: 存储边缘特征块的列表（在代码中被注释掉）。
     - `pool_blocks`: 存储池化块的列表。
     - `latent_size`: 输出特征的维度。
     - `normalization`: 层归一化层。
   - **初始化过程**:
     - 根据消息传递步数，初始化图神经网络块和池化块。
2. **前向传播 (`forward` 方法)**:
   - **输入**:
     - `latent_graph`: 输入的图数据。
     - `normalized_adj_mat`: 归一化的邻接矩阵（未在代码中使用）。
   - **输出**:
     - `x`: 存储每一步处理后的节点特征的列表。
     - `pos`: 存储每一步处理后的节点位置信息的列表。
   - **处理过程**:
     - 初始化列表 `x`, `pos`, 和 `new`。
     - 遍历图神经网络块和池化块：
       - 如果当前图的节点数大于 `min_nodes`，则进行消息传递和池化操作：
         - 使用图神经网络块处理图数据。
         - 使用池化块更新边缘特征。
         - 获取粗节点索引。
         - 提取粗节点的特征和位置信息。
         - 使用 `graph_connectivity` 函数更新边的索引和权重。
         - 更新归一化后的边权重。
         - 更新 `latent_graph` 为新的图数据。
       - 否则，仅使用图神经网络块处理图数据，并将结果存储在 `new` 列表中。
     - 如果 `new` 列表不为空，将最后一个图数据添加到 `x` 列表中。
     - 返回 `x` 和 `pos`。



In [9]:
class LazyMLP(nn.Module):
    def __init__(self, output_sizes):
        super().__init__()
        num_layers = len(output_sizes)
        self._layers_ordered_dict = OrderedDict()
        for index, output_size in enumerate(output_sizes):
            self._layers_ordered_dict["linear_" + str(index)] = nn.LazyLinear(output_size)
            if index < (num_layers - 1):
                self._layers_ordered_dict["relu_" + str(index)] = nn.ReLU()
        self.layers = nn.Sequential(self._layers_ordered_dict)

    def forward(self, input):
        input = input.to(device)
        y = self.layers(input)
        return y

###  `LazyMLP` 类

**内容:**

`LazyMLP` 是一个继承自 `nn.Module` 的 PyTorch 模块，它实现了一个“懒惰”的多层感知器（MLP）。所谓“懒惰”是指它的输入维度不需要在初始化时指定，而是在第一次前向传播时根据输入数据自动推断（通过 `nn.LazyLinear` 层实现）。

**参数:**

*   `widths`: 一个列表，指定MLP每一层的输出维度。列表的第一个元素是输入维度（可以是任意值，因为会忽略或推断），后续元素定义了每一层的宽度，最后一个元素是最终的输出维度。例如 `[input_dim, 128, 64, output_dim]`。

**内部结构:**

*   `self.layers`: 一个 `nn.Sequential` 容器，包含一系列 `nn.LazyLinear` 层和 `nn.ReLU` 激活函数层（除了最后一层）。

**方法:**

*   `__init__(self, widths)`: 初始化方法，根据 `widths` 列表构建MLP层。
*   `forward(self, input)`: 前向传播方法，接收输入数据，通过MLP层进行计算，并返回输出。

In [10]:
class Encoder(nn.Module):

    def __init__(self, make_mlp, latent_size):
        super().__init__()
        self._make_mlp = make_mlp
        self._latent_size = latent_size
        self.node_model = self._make_mlp(latent_size)
        self.mesh_edge_model = self._make_mlp(latent_size)

    def forward(self, graph):
        node_latents = self.node_model(graph.x)
        edge_latent = self.mesh_edge_model(graph.edge_attr)
        graph.x=node_latents
        graph.edge_attr=edge_latent
        return graph

class Decoder(nn.Module):

    def __init__(self, make_mlp, output_size):
        super().__init__()
        self.model = make_mlp(output_size)

    def forward(self,node_features):
        return self.model(node_features)

### `Encoder` 类
`Encoder` 类负责将图中的节点和边特征编码为潜在特征（latent features）。它的结构如下：
- **`__init__`**: 初始化方法，创建两个 MLP 模型，分别用于处理节点特征和边特征。
- **`forward`**: 前向传播方法，将输入图的节点和边特征分别传递给对应的 MLP 模型，并更新图的特征。

### `Decoder` 类
`Decoder` 类负责将潜在特征解码为输出特征。它的结构如下：
- **`__init__`**: 初始化方法，创建一个 MLP 模型用于处理节点特征。
- **`forward`**: 前向传播方法，将输入的节点特征传递给 MLP 模型，并返回输出。

In [11]:
class EncodeProcessDecode(nn.Module):
    """Encode-Process-Decode GraphNet model."""
    def __init__(self,
                 output_size,
                 latent_size,
                 num_layers,
                 message_passing_aggregator, message_passing_steps,
                 ):
        super().__init__()
        self._latent_size = latent_size
        self._output_size = output_size
        self._num_layers = num_layers
        self._message_passing_steps = message_passing_steps
        self._message_passing_aggregator = message_passing_aggregator   
        self.encoder = Encoder(make_mlp=self._make_mlp, latent_size=self._latent_size)
        self.processor = Processor(make_mlp=self._make_mlp, output_size=self._latent_size,
                                   message_passing_steps=self._message_passing_steps,
                                   message_passing_aggregator=self._message_passing_aggregator,
                                   stochastic_message_passing_used=False)
        self.post_processor=self._make_mlp(self._latent_size)                           
        self.decoder = Decoder(make_mlp=functools.partial(self._make_mlp, layer_norm=False),
                               output_size=self._output_size)

    def _make_mlp(self, output_size, layer_norm=True):
        """Builds an MLP."""
        widths = [self._latent_size] * self._num_layers + [output_size]
        network = LazyMLP(widths)
        if layer_norm:
            network = nn.Sequential(network, nn.LayerNorm(normalized_shape=widths[-1]))
        return network

    def spa_compute(self,x,p):
        j=len(x)-1
        node_features=x[j].x
        x_pos=x[j].pos
        for k in range(1,j+1):
            pos=p[-k]
            feature=knn_interpolate(node_features,pos,x[-(k+1)].pos)  
            node_features=x[-(k+1)].x+feature                  
            node_features=self.post_processor(node_features)
                    
        return node_features     
    def forward(self, graph):
        latent_graph = self.encoder(graph)
        x,p= self.processor(latent_graph)
        node_features=self.spa_compute(x,p)
        return self.decoder(node_features)

### 定义 `EncodeProcessDecode` 类
`EncodeProcessDecode` 是主模型类，它结合了编码器、处理器和解码器。它的结构如下：
- **`__init__`**: 初始化方法，创建编码器、处理器和解码器实例，并定义一些参数。
  - `encoder`: 编码器实例，用于将节点和边特征编码为潜在特征。
  - `processor`: 处理器实例，用于在图上进行消息传递和特征更新。
  - `post_processor`: 后处理 MLP，用于进一步处理节点特征。
  - `decoder`: 解码器实例，用于将潜在特征解码为输出特征。
- **`_make_mlp`**: 辅助方法，用于构建 MLP 网络。
- **`spa_compute`**: 特殊的前向传播方法，用于处理节点特征。它使用 `knn_interpolate` 进行插值，并逐步更新节点特征。
- **`forward`**: 前向传播方法，将输入图传递给编码器、处理器和 `spa_compute` 方法，最后通过解码器生成输出。

In [12]:
import os
import sys
import logging
def logger_setup(log_path):
    # set log configuration
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)
    # console_output_handler = logging.StreamHandler(sys.stdout)
    # console_output_handler.setLevel(logging.INFO)
    file_log_handler = logging.FileHandler(filename=log_path, mode='w', encoding='utf-8')
    file_log_handler.setLevel(logging.INFO)
    formatter = logging.Formatter(fmt='%(asctime)s - %(message)s')
    # console_output_handler.setFormatter(formatter)
    file_log_handler.setFormatter(formatter)
    # root_logger.addHandler(console_output_handler)
    root_logger.addHandler(file_log_handler)
    return root_logger

### 日志记录器
1. **创建日志记录器**：
   - 使用 `logging.getLogger()` 获取根日志记录器，并将其日志级别设置为 `INFO`。
2. **配置文件日志处理器**：
   - 创建一个 `FileHandler`，将日志写入指定的文件路径（`log_path`），模式为写入模式（`'w'`），编码为 `utf-8`。
   - 将日志级别设置为 `INFO`。
3. **设置日志格式**：
   - 使用 `logging.Formatter` 设置日志格式，格式为 `'%(asctime)s - %(message)s'`，即时间戳和日志消息。
4. **添加日志处理器**：
   - 将文件日志处理器添加到根日志记录器中。
5. **返回日志记录器**：
   - 返回配置好的根日志记录器。

# Airfoil数据集

## 数据集处理

In [15]:
import os
import pickle
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import math,copy
from os import PathLike
from typing import Sequence, Dict, Union, Tuple, List
import torch
from torch._six import string_classes
import collections.abc as container_abcs
from torch_geometric.data import Data, Batch, Dataset
from torch_geometric.loader import DataLoader
from scipy.spatial.qhull import Delaunay
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision import utils
from torchvision.utils import make_grid

In [16]:
SU2_SHAPE_IDS = {
    'line': 3,
    'triangle': 5,
    'quad': 9,
}

def get_mesh_graph(mesh_filename: Union[str, PathLike],
                   dtype: np.dtype = np.float32
                   ) -> Tuple[np.ndarray, np.ndarray, List[List[List[int]]], Dict[str, List[List[int]]]]:
    def get_rhs(s: str) -> str:
        return s.split('=')[-1]

    marker_dict = {}
    with open(mesh_filename) as f:
        for line in f:
            if line.startswith('NPOIN'):
                num_points = int(get_rhs(line))
                mesh_points = [[float(p) for p in f.readline().split()[:2]]
                               for _ in range(num_points)]
                nodes = np.array(mesh_points, dtype=dtype)

            if line.startswith('NMARK'):
                num_markers = int(get_rhs(line))
                for _ in range(num_markers):
                    line = f.readline()
                    assert line.startswith('MARKER_TAG')
                    marker_tag = get_rhs(line).strip()
                    num_elems = int(get_rhs(f.readline()))
                    marker_elems = [[int(e) for e in f.readline().split()[-2:]]
                                    for _ in range(num_elems)]
                    # marker_dict[marker_tag] = np.array(marker_elems, dtype=np.long).transpose()
                    marker_dict[marker_tag] = marker_elems

            if line.startswith('NELEM'):
                edges = []
                triangles = []
                quads = []
                num_edges = int(get_rhs(line))
                for _ in range(num_edges):
                    elem = [int(p) for p in f.readline().split()]
                    if elem[0] == SU2_SHAPE_IDS['triangle']:
                        n = 3
                        triangles.append(elem[1:1+n])
                    elif elem[0] == SU2_SHAPE_IDS['quad']:
                        n = 4
                        quads.append(elem[1:1+n])
                    else:
                        raise NotImplementedError
                    elem = elem[1:1+n]
                    edges += [[elem[i], elem[(i+1) % n]] for i in range(n)]
                edges = np.array(edges, dtype=np.long).transpose()
                # triangles = np.array(triangles, dtype=np.long)
                # quads = np.array(quads, dtype=np.long)
                elems = [triangles, quads]


    return nodes, edges, elems, marker_dict

### 定义常量
定义了一个常量 `SU2_SHAPE_IDS`，用于标识不同形状的网格元素（如线、三角形、四边形）。
### 辅助函数 `get_mesh_graph`
这个函数用于从 `.su2` 文件中读取网格数据，并返回节点坐标、边索引、元素列表和标记字典。

In [17]:
class MeshAirfoilDataset(Dataset):
    def __init__(self, root, mode='train'):
        self.mode = mode
        self.data_dir = Path(root) / ('outputs_' + mode)
        self.file_list = os.listdir(self.data_dir)
        self.len = len(self.file_list)

        self.mesh_graph = get_mesh_graph(Path(root) / 'mesh_fine.su2')

        # either [maxes, mins] or [means, stds] from data for normalization
        # with open(self.data_dir / 'train_mean_std.pkl', 'rb') as f:
        with open(self.data_dir.parent / 'train_max_min.pkl', 'rb') as f:
            self.normalization_factors = pickle.load(f)

        self.nodes = torch.from_numpy(self.mesh_graph[0])
        self.edges = torch.from_numpy(self.mesh_graph[1])
        self.elems_list = self.mesh_graph[2]
        self.marker_dict = self.mesh_graph[3]
        self.node_markers = self.nodes.new_full((self.nodes.shape[0], 1), fill_value=-1)
        for i, (marker_tag, marker_elems) in enumerate(self.marker_dict.items()):
            for elem in marker_elems:
                self.node_markers[elem[0]] = i
                self.node_markers[elem[1]] = i

        super().__init__(root)
        
    def len(self):
        return self.len

    def get(self, idx):
        with open(self.data_dir / self.file_list[idx], 'rb') as f:
            fields = pickle.load(f)
        fields = self.preprocess(fields)

        aoa, reynolds, mach = self.get_params_from_name(self.file_list[idx])
        aoa = aoa
        aoa = torch.from_numpy(aoa)
        mach_or_reynolds = mach if reynolds is None else reynolds
        mach_or_reynolds = torch.from_numpy(mach_or_reynolds)

        norm_aoa = aoa / 10
        norm_mach_or_reynolds = mach_or_reynolds if reynolds is None else (mach_or_reynolds - 1.5e6) / 1.5e6

        nodes = torch.cat([
            self.nodes,
            norm_aoa.unsqueeze(0).repeat(self.nodes.shape[0], 1),
            norm_mach_or_reynolds.unsqueeze(0).repeat(self.nodes.shape[0], 1),
            self.node_markers
        ], dim=-1)

        data = Data(x=nodes, y=fields, edge_index=self.edges,pos=self.nodes)

        sender=data.x[data.edge_index[0]]
        receiver=data.x[data.edge_index[1]]
        relation_pos=sender[:,0:2]-receiver[:,0:2]
        post=torch.norm(relation_pos,p=2,dim=1,keepdim=True)
        data.edge_attr=post
        
        #####归一化边特征
        std_epsilon=torch.tensor([1e-8])
        a=torch.mean(data.edge_attr,axis=0,dtype=torch.float32)
        b=data.edge_attr.std(dim=0)
        b=torch.maximum(b,std_epsilon)
        data.edge_attr=(data.edge_attr-a)/b
        
        data.aoa = aoa
        data.norm_aoa = norm_aoa
        data.mach_or_reynolds = mach_or_reynolds
        data.norm_mach_or_reynolds = norm_mach_or_reynolds
        return data

    def preprocess(self, tensor_list, stack_output=True):
        data_max, data_min = self.normalization_factors
        normalized_tensors = []
        for i in range(len(tensor_list)):
            normalized = (tensor_list[i] - data_min[i]) / (data_max[i] - data_min[i]) * 2 - 1
            if type(normalized) is np.ndarray:
                normalized = torch.from_numpy(normalized)
            normalized_tensors.append(normalized)
        if stack_output:
            normalized_tensors = torch.stack(normalized_tensors, dim=1)
        return normalized_tensors

    def _download(self):
        pass

    def _process(self):
        pass

    @staticmethod
    def get_params_from_name(filename):
        s = filename.rsplit('.', 1)[0].split('_')
        aoa = np.array(s[s.index('aoa') + 1])[np.newaxis].astype(np.float32)
        reynolds = s[s.index('re') + 1]
        reynolds = np.array(reynolds)[np.newaxis].astype(np.float32) if reynolds != 'None' else None
        mach = np.array(s[s.index('mach') + 1])[np.newaxis].astype(np.float32)
        return aoa, reynolds, mach

### 数据集类 `MeshAirfoilDataset`
这是一个继承自 `torch_geometric.data.Dataset` 的类，用于加载和处理空气动力学网格数据。主要功能包括：
- **初始化**：读取网格文件，加载归一化因子，初始化节点、边、元素列表和标记字典。
- **`__len__`**：返回数据集的长度。
- **`get`**：根据索引加载并预处理数据，包括读取物理场数据、归一化处理、添加物理参数到节点特征、计算边特征并归一化。
- **`preprocess`**：对物理场数据进行归一化处理。
- **`_download` 和 `_process`**：空方法，用于满足 `Dataset` 类的接口要求。
- **`get_params_from_name`**：从文件名中提取物理参数（攻角、雷诺数、马赫数）。

In [18]:
def log_images(nodes, pred, true, batch, elems_list, mode, log_idx=0,iterate=0,file='field.png'):
        inds = batch == log_idx
        nodes = nodes[inds]
        pred = pred[inds] 
        true = true[inds]
        for field in range(pred.shape[1]):
            true_img = plot_field(nodes, elems_list, true[:, field],
                                  title='true')
            true_img = ToTensor()(true_img)
            min_max = (true[:, field].min().item(), true[:, field].max().item())

            pred_img = plot_field(nodes, elems_list, pred[:, field],
                                  title='pred',clim=min_max)  #clim=min_max 
            pred_img=ToTensor()(pred_img)
            imgs=[pred_img,true_img]  
            grid = make_grid(torch.stack(imgs), padding=0) 
            out_file=file+f'{field}'
            utils.save_image(grid,out_file+'_field.png')  
        # 释放内存
        del true_img, pred_img, grid, imgs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

def plot_field(nodes, elems_list, field, contour=False, clim=None, zoom=True,
               get_array=True, out_file=None, show=False, title=''):
    elems_list = sum(elems_list, [])
    tris, _ = quad2tri(elems_list)
    tris = np.array(tris)
    x, y = nodes[:, :2].t().detach().cpu().numpy()
    field = field.detach().cpu().numpy()
    fig = plt.figure(dpi=800)
    if contour:
        plt.tricontourf(x, y, tris, field)
    else:
        plt.tripcolor(x, y, tris, field)
    if clim:
        plt.clim(*clim)
    plt.colorbar()
    if zoom:
        plt.xlim(left=-0.5, right=1.5)
        plt.ylim(bottom=-1.0, top=1.0)
    if title:
        plt.title(title)

    if out_file is not None:
        plt.savefig(out_file)
        plt.close()

    if show:
         plt.show()
        #raise NotImplementedError

    if get_array:
        fig.canvas.draw()
        a = np.fromstring(fig.canvas.tostring_rgb(),
                          dtype=np.uint8, sep='')
        a = a.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close()
        return a
    
def quad2tri(elems):
    new_elems = []
    new_edges = []
    for e in elems:
        if len(e) <= 3:
            new_elems.append(e)
        else:
            new_elems.append([e[0], e[1], e[2]])
            new_elems.append([e[0], e[2], e[3]])
            new_edges.append(torch.tensor(([[e[0]], [e[2]]]), dtype=torch.long))
    new_edges = torch.cat(new_edges, dim=1) if new_edges else torch.tensor([], dtype=torch.long)
    return new_elems, new_edges

### 辅助函数 `log_images`
用于将预测和真实的物理场数据可视化并保存为图片。
### 辅助函数 `plot_field`
用于绘制物理场数据，支持等高线图和颜色填充图，并可以调整显示范围和保存图片。
### 辅助函数 `quad2tri`
将四边形网格元素转换为三角形网格元素，并返回新的元素列表和边索引。

In [20]:
dataset=[]
train_loader=MeshAirfoilDataset('/home/ma-user/work/data/NACA0012_interpolate/',mode='train')
for i in range(train_loader.len):
    data=train_loader.get(i)
    dataset.append(data)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# 从loader中获取一份数据
data_batch = next(iter(loader))

# 查看数据的形状
print("Data batch shape:", data_batch)

Data batch shape: DataBatch(x=[26736, 5], edge_index=[2, 105344], y=[26736, 3], pos=[26736, 2], edge_attr=[105344, 1], aoa=[4], norm_aoa=[4], mach_or_reynolds=[4], norm_mach_or_reynolds=[4], batch=[26736], ptr=[5])


## 训练

In [ ]:
from tqdm import tqdm

device = torch.device("cuda:0")
summaryWriter=SummaryWriter("/home/ma-user/work/logs/log1")
min_nodes = 2000

model=EncodeProcessDecode(output_size=3,
                 latent_size=128,
                 num_layers=2,
                 message_passing_aggregator='sum', message_passing_steps=6).to(device)
optimizers = optim.Adam(model.parameters(), lr=0.0005)#weight_decay=5e-4
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizers, 0.1 + 1e-6, last_epoch=-1)
criterion =torch.nn.MSELoss().to(device)
root_logger = logger_setup(os.path.join('/home/ma-user/work/', 'logairfoil.log'))

model.train()
root_logger.info("===========start train===========") 
loss_history=[]
for epoch in range(500):
    sum_loss = 0
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{500}")  # 添加训练进度条
    for batch in pbar:
        batch = batch.to(device)
        truefield = batch.y
        prefield = model(batch)
        #if epoch % 9 == 0:
            #log_images(batch.pos, prefield, truefield, batch.batch, train_loader.elems_list, 'train')
        mes_loss = criterion(prefield, truefield)
        optimizers.zero_grad()
        mes_loss.backward()
        optimizers.step()
        loss = mes_loss
        sum_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})  # 更新进度条显示的损失值
    loss_history.append(sum_loss / len(loader))
    sum_loss1 = (sum_loss) / len(loader)
    summaryWriter.add_scalar("loss", math.sqrt(sum_loss1), epoch)
    root_logger.info("        trajectory_loss")
    root_logger.info("        " + str(sum_loss1))
    if (epoch == 60) | (epoch == 100) | (epoch == 140) | (epoch == 170):
        scheduler.step()

torch.save(model.state_dict(), '/home/ma-user/work/result/modelairfoil.pkl')

/home/ma-user/anaconda3/envs/PyTorch-1.10.2/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Epoch 100/500:  86%|████████▌ | 36/42 [01:12<00:12,  2.02s/it, loss=0.000143]

## 测试

In [17]:
from tqdm import tqdm

# 该部分生成图像时可能因为内存原因中断，请手动拆分为两部分进行生成
device = torch.device("cuda:0")
summaryWriter=SummaryWriter("/home/ma-user/work/logs/log1")

model=EncodeProcessDecode(output_size=3,
                 latent_size=128,
                 num_layers=2,
                 message_passing_aggregator='sum', message_passing_steps=6).to(device)
model.load_state_dict(torch.load('/home/ma-user/work/result/modelairfoil.pkl'))      
min_nodes = 2000

dataset_test=[]
test_loader=MeshAirfoilDataset('/home/ma-user/work/data/NACA0012_interpolate/',mode='test')
for i in range(test_loader.len):
    data=test_loader.get(i)
    dataset_test.append(data)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=True)

optimizers = optim.Adam(model.parameters(), lr=0.0005)#weight_decay=5e-4
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizers, 0.1 + 1e-6, last_epoch=-1)
criterion =torch.nn.MSELoss().to(device)
root_logger = logger_setup(os.path.join('/home/ma-user/work/', 'logairfoil.log'))

model.eval()
root_logger.info("===========start test===========") 
with torch.no_grad():
    sum_loss = 0
    pbar = tqdm(test_loader, desc="Testing")  # 添加测试进度条
    pic_id = 0
    for batch in pbar:
        batch = batch.to(device)
        truefield = batch.y
        prefield = model(batch)
        pic_name = 'result_pic/test_pic' + str(pic_id) + '_'
        pic_id += 1
        if pic_id >= 0:
            log_images(batch.pos, prefield, truefield, batch.batch, train_loader.elems_list, 'test', file=pic_name)
        mes_loss = criterion(prefield, truefield)
        loss = mes_loss.cpu()
        loss = np.sqrt(loss)
        sum_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})  # 更新进度条显示的损失值
sum_loss1 = sum_loss / (len(test_loader))
root_logger.info("        trajectory_loss")
root_logger.info("        " + str(sum_loss1))

/home/ma-user/anaconda3/envs/PyTorch-1.10.2/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Testing: 100%|██████████| 63/63 [07:11<00:00,  6.84s/it, loss=0.0216] 


# Cylinder

## 数据处理

In [14]:
import os
import pickle
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import math,copy
from os import PathLike
from typing import Sequence, Dict, Union, Tuple, List
import torch
from torch._six import string_classes
import collections.abc as container_abcs
from torch_geometric.data import Data, Batch, Dataset
from torch_geometric.loader import DataLoader
from scipy.spatial.qhull import Delaunay
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from torchvision import utils
from torchvision.utils import make_grid

In [15]:
import re
SU2_SHAPE_IDS = {
    'line': 3,
    'triangle': 5,
    'quad': 9,
}
def get_mesh_graph(mesh_filename: Union[str, PathLike],
                   dtype: np.dtype = np.float32
                   ) -> Tuple[np.ndarray, np.ndarray, List[List[List[int]]], Dict[str, List[List[int]]]]:
    def get_rhs(s: str) -> str:
        return s.split('=')[-1]

    marker_dict = {}
    with open(mesh_filename) as f:
        for line in f:
            if line.startswith('NPOIN'):
                num_points = int(get_rhs(line))
                mesh_points = [[float(p) for p in f.readline().split()[:2]]
                               for _ in range(num_points)]
                nodes = np.array(mesh_points, dtype=dtype)

            if line.startswith('NMARK'):
                num_markers = int(get_rhs(line))
                for _ in range(num_markers):
                    line = f.readline()
                    assert line.startswith('MARKER_TAG')
                    marker_tag = get_rhs(line).strip()
                    num_elems = int(get_rhs(f.readline()))
                    marker_elems = [[int(e) for e in f.readline().split()[-2:]]
                                    for _ in range(num_elems)]
                    marker_dict[marker_tag] = marker_elems

            if line.startswith('NELEM'):
                edges = []
                triangles = []
                quads = []
                num_edges = int(get_rhs(line))
                for _ in range(num_edges):
                    elem = [int(p) for p in f.readline().split()]
                    if elem[0] == SU2_SHAPE_IDS['triangle']:
                        n = 3
                        triangles.append(elem[1:1+n])
                    elif elem[0] == SU2_SHAPE_IDS['quad']:
                        n = 4
                        quads.append(elem[1:1+n])
                    else:
                        raise NotImplementedError
                    elem = elem[1:1+n]
                    edges += [[elem[i], elem[(i+1) % n]] for i in range(n)]
                edges = np.array(edges, dtype=np.long).transpose()
                elems = [triangles, quads]


    return nodes, edges, elems, marker_dict

class MeshcylinderDataset(Dataset):
    def __init__(self, root, mode='train'):
        self.mode = mode
        self.data_dir = Path(root) / (mode)
        self.file_list = [f for f in os.listdir(self.data_dir) if '.ipynb_checkpoints' not in f]
        self.len = len(self.file_list)

        self.mesh_graph = get_mesh_graph(Path(root) / 'cylinder.su2')

        self.normalization_factors =torch.tensor([[978.6001,  48.9258,  24.8404],
        [-692.3159,   -6.9950,  -24.8572]])
        self.nodes = torch.from_numpy(self.mesh_graph[0])
        self.meshnodes=self.mesh_graph[0]
        self.edges = torch.from_numpy(self.mesh_graph[1])
        self.elems_list = self.mesh_graph[2]
        self.marker_dict = self.mesh_graph[3]
        self.bounder=[]
        self.node_markers = self.nodes.new_full((self.nodes.shape[0], 1), fill_value=-1)
        for i, (marker_tag, marker_elems) in enumerate(self.marker_dict.items()):
            for elem in marker_elems:
                self.node_markers[elem[0]] = i
                self.node_markers[elem[1]] = i

        super().__init__(root)
        
    def __len__(self):
        return self.len

    def get(self, idx):
        with open(self.data_dir/self.file_list[idx],'r') as f:
            field=[]
            pos=[]
            i=1
            for lines in f.readlines():
                if not i: 
                    lines=lines.rstrip('\n')
                    lines_pos=lines.split(',')[1:3]
                    lines_field=lines.split(',')[3:]
                    #for i in range(len(lines)):
                    #    a=float(lines[i])
                    numbers_float =list(eval(i) for i in lines_pos)
                    array=np.array(numbers_float,np.float32)
                    a=torch.from_numpy(array) 
                    pos.append(a)
                    numbers_float =list(eval(i) for i in lines_field)
                    array=np.array(numbers_float,np.float32)
                    a=torch.from_numpy(array) 
                    field.append(a)
                i=0
        field=torch.stack(field,axis=0)
        pos= torch.stack(pos,axis=0)       
        indexlist=[]
        for i in range(self.meshnodes.shape[0]):
            b=torch.from_numpy(self.meshnodes[i:(i+1)])
            b=torch.squeeze(b)
            index=torch.nonzero(torch.sum((pos==b),axis=1,dtype=torch.float32)==pos.shape[1])
            indexlist.append(index)    
        indexlist=torch.stack(indexlist,axis=0)
        indexlist=torch.squeeze(indexlist)
        fields=field[indexlist]

        velocity= self.get_params_from_name(self.file_list[idx])
        aoa = torch.from_numpy(velocity)

        norm_aoa = (aoa/40)
        nodes = torch.cat([
            self.nodes,
            norm_aoa.unsqueeze(0).repeat(self.nodes.shape[0], 1),
            self.node_markers
        ], dim=-1).to(torch.float32)

        data = Data(x=nodes, y=fields, edge_index=self.edges,pos=self.nodes,velocity=aoa)

        sender=data.x[data.edge_index[0]]
        receiver=data.x[data.edge_index[1]]
        relation_pos=sender[:,0:2]-receiver[:,0:2]
        post=torch.norm(relation_pos,p=2,dim=1,keepdim=True)
        data.edge_attr=post
        # 边特征归一化
        std_epsilon=torch.tensor([1e-8])
        a=torch.mean(data.edge_attr,axis=0,dtype=torch.float32)
        b=data.edge_attr.std(dim=0)
        b=torch.maximum(b,std_epsilon)
        data.edge_attr=(data.edge_attr-a)/b
        
        a=torch.mean(data.y,axis=0,dtype=torch.float32)
        b=data.y.std(dim=0)
        b=torch.maximum(b,std_epsilon)
        data.y=(data.y-a)/b
        
        data.norm_max = a
        data.norm_min = b
        
        # y_min = torch.min(data.y, dim=0).values
        # y_max = torch.max(data.y, dim=0).values
        # data.y = 2 * (data.y - y_min) / (y_max - y_min) - 1
        # data.norm_min = y_min
        # data.norm_max = y_max
        
        with open('/home/ma-user/work/data/cylinderdata/bounder','r') as f:
            field=[]
            pos=[]
            i=1
            for lines in f.readlines():
                if not i: 
                    lines=lines.rstrip('\n')
                    lines_pos=lines.split(',')[1:3]
                    lines_field=lines.split(',')[3:]
                    numbers_float =list(eval(i) for i in lines_pos)
                    array=np.array(numbers_float,np.float32)
                    a=torch.from_numpy(array) 
                    pos.append(a)
                    numbers_float =list(eval(i) for i in lines_field)
                    array=np.array(numbers_float,np.float32)
                    a=torch.from_numpy(array) 
                    field.append(a)
                i=0
        field=torch.stack(field,axis=0)
        pos= torch.stack(pos,axis=0)       
        indexlist=[]
        for i in range(pos.shape[0]):
            b=pos[i:(i+1)]
            b=torch.squeeze(b)
            index=torch.nonzero(torch.sum((self.nodes==b),dim=1,dtype=torch.float32)==self.nodes.shape[1])
            indexlist.append(index) 
        indexlist=torch.stack(indexlist,dim=0)
        indexlist=torch.squeeze(indexlist)
        self.bounder=indexlist
        return data
    
    @staticmethod
    def get_params_from_name(filename):
        s = filename.rsplit('.', 1)[0]
        reynolds = np.array(s[13:])[np.newaxis].astype(np.float32)
        return reynolds

In [22]:
def log_images(nodes, pred, true, batch, elems_list, mode, log_idx=0,iterate=0,file='field.png'):
        inds = batch == log_idx
        nodes = nodes[inds]
        pred = pred[inds] 
        true = true[inds]
        for field in range(pred.shape[1]):
            # min_max = (-0.8, 0.8)

            true_img = plot_field(nodes, elems_list, true[:, field],
                                  title='true')
            true_img = ToTensor()(true_img)
            
            min_max = (true[:, field].min().item(), true[:, field].max().item())

            pred_img = plot_field(nodes, elems_list, pred[:, field],
                                  title='pred',clim=min_max)  #clim=min_max 
            pred_img=ToTensor()(pred_img)
            
            imgs=[pred_img,true_img]  
            grid = make_grid(torch.stack(imgs), padding=0) 
            out_file=file+f'{field}'
            utils.save_image(grid,out_file+'_field.png')  
        # 释放内存
        del true_img, pred_img, grid, imgs
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

def plot_field(nodes, elems_list, field, contour=False, clim=None, zoom=True,
               get_array=True, out_file=None, show=False, title=''):
    elems_list = sum(elems_list, [])
    tris, _ = quad2tri(elems_list)
    tris = np.array(tris)
    x, y = nodes[:, :2].t().detach().cpu().numpy()
    field = field.detach().cpu().numpy()
    fig = plt.figure(dpi=800)
    if contour:
        plt.tricontourf(x, y, tris, field)
    else:
        plt.tripcolor(x, y, tris, field)
    if clim:
        plt.clim(*clim)
    plt.colorbar()
    if zoom:
        plt.xlim(left=-3.0, right=3.0)
        plt.ylim(bottom=-3.0, top=3.0)
    if title:
        plt.title(title)

    if out_file is not None:
        plt.savefig(out_file)
        plt.close()

    if show:
         plt.show()
        #raise NotImplementedError

    if get_array:
        fig.canvas.draw()
        a = np.fromstring(fig.canvas.tostring_rgb(),
                          dtype=np.uint8, sep='')
        a = a.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close()
        return a
    
def quad2tri(elems):
    new_elems = []
    new_edges = []
    for e in elems:
        if len(e) <= 3:
            new_elems.append(e)
        else:
            new_elems.append([e[0], e[1], e[2]])
            new_elems.append([e[0], e[2], e[3]])
            new_edges.append(torch.tensor(([[e[0]], [e[2]]]), dtype=torch.long))
    new_edges = torch.cat(new_edges, dim=1) if new_edges else torch.tensor([], dtype=torch.long)
    return new_elems, new_edges

In [17]:
dataset=[]
train_loader=MeshcylinderDataset('/home/ma-user/work/data/cylinderdata',mode='train')
for i in range(train_loader.len): 
    data=train_loader.get(i)
    dataset.append(data)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

dataset_test = []
test_loader_=MeshcylinderDataset('/home/ma-user/work/data/cylinderdata',mode='test')
for i in range(test_loader_.len):
    data = test_loader_.get(i)
    dataset_test.append(data)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=True)

In [91]:
data = train_loader.get(0)
print(data.x)
print(data.y)
print(data.velocity)

tensor([[-10.0000, -20.0000,   0.9375,   0.0000],
        [ -8.2759, -20.0000,   0.9375,   0.0000],
        [ -6.5517, -20.0000,   0.9375,   0.0000],
        ...,
        [  1.8668,  -0.4482,   0.9375,  -1.0000],
        [  1.9014,  -0.3012,   0.9375,  -1.0000],
        [  1.9139,  -0.1506,   0.9375,  -1.0000]])
tensor([[ 2.8666e-01,  7.6498e-01,  4.5367e-04],
        [ 2.8341e-01,  7.6681e-01,  4.5367e-04],
        [ 2.7564e-01,  7.7028e-01,  4.5367e-04],
        ...,
        [-8.6706e-01, -1.6042e+00, -1.7251e-01],
        [-8.6152e-01, -1.6480e+00, -1.2177e-01],
        [-8.5795e-01, -1.6721e+00, -6.3587e-02]])
tensor([37.5000])


## 训练

In [ ]:
from tqdm import tqdm

device = torch.device("cuda:0")

model=EncodeProcessDecode(output_size=3,
                 latent_size=128,
                 num_layers=2,
                 message_passing_aggregator='sum', message_passing_steps=6).to(device)

optimizers = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizers, gamma=0.1)
criterion =torch.nn.MSELoss().to(device)

pre_loss = 100
min_nodes = 20000

for epoch in range(500):
    
    model.train()
    pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{500}")  # 添加训练进度条
    for batch in pbar:
        batch = batch.to(device)
        truefield = batch.y
        batch.x = batch.x.to(torch.float32)
        batch.velocity = batch.velocity.to(torch.float32)
        batch.edge_attr = batch.edge_attr.to(torch.float32)
        prefield = model(batch)
        
        mes_loss = criterion(prefield, truefield)
        optimizers.zero_grad()
        mes_loss.backward()
        optimizers.step()
        loss = mes_loss
        pbar.set_postfix({'loss': loss.item()}) 
        
    sum_loss = 0
    model.eval()
    test_tqdm = tqdm(test_loader, desc="Testing")
    with torch.no_grad():
        for batch in test_tqdm:
            batch = batch.to(device)
            truefield = batch.y
            batch.x = batch.x.to(torch.float32)
            batch.velocity = batch.velocity.to(torch.float32)
            batch.edge_attr = batch.edge_attr.to(torch.float32)
            prefield = model(batch)

            test_mes_loss = criterion(prefield, truefield)
            test_loss = test_mes_loss.cpu()
            test_loss = np.sqrt(test_loss)
            sum_loss += test_loss.item()
            test_tqdm.set_postfix({'loss': test_loss.item()}) 
            
    if((epoch==60)|(epoch==100)|(epoch==140)|(epoch==170)): 
        scheduler.step()
        
    if sum_loss < pre_loss:
        pre_loss = sum_loss
        torch.save(model.state_dict(), '/home/ma-user/work/result/modelcylinder.pkl')

## 测试

In [ ]:
from tqdm import tqdm

device = torch.device("cuda:0")
summaryWriter=SummaryWriter("logs/log2")

model=EncodeProcessDecode(output_size=3,
                 latent_size=128,
                 num_layers=2,
                 message_passing_aggregator='sum', message_passing_steps=6).to(device)
model.load_state_dict(torch.load('/home/ma-user/work/result/modelcylinder.pkl'))  
# model.load_state_dict(torch.load('/home/ma-user/work/result/modelairfoil.pkl'))  

min_nodes = 20000

optimizers = optim.Adam(model.parameters(), lr=0.0001)#weight_decay=5e-4
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizers, 0.1 + 1e-6, last_epoch=-1)
criterion =torch.nn.MSELoss().to(device)
root_logger = logger_setup(os.path.join('/home/ma-user/work', 'log_cylinder_test.log'))

root_logger.info("===========start test===========")

model.eval()
with torch.no_grad():
    sum_loss = 0
    # 使用 tqdm 添加进度条
    pic_id = 0
    
    test_tqdm = tqdm(test_loader, desc="Testing")
    for batch in test_tqdm:
        batch = batch.to(device)
        truefield = batch.y
        batch.x = batch.x.to(torch.float32)
        batch.velocity = batch.velocity.to(torch.float32)
        batch.edge_attr = batch.edge_attr.to(torch.float32)
        prefield = model(batch)
        
        pic_name = 'cylinder_result_pic/test_pic' + str(pic_id) + '_'
        pic_id += 1
        if pic_id >= 0:
            log_images(batch.pos, prefield, truefield, batch.batch, train_loader.elems_list, 'test', file=pic_name)

        # print(prefield, truefield)
        mes_loss = criterion(prefield, truefield)
        loss = mes_loss.cpu()
        loss = np.sqrt(loss)
        sum_loss += loss.item()
        
        test_tqdm.set_postfix({'loss': loss.item()})

Testing:  82%|████████▏ | 9/11 [02:11<00:28, 14.17s/it, loss=0.0501]

### 数据后处理
打包数据与上传obs

In [104]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    """
    压缩指定的文件夹到ZIP文件中。

    :param folder_path: 要压缩的文件夹路径
    :param zip_path: 生成的ZIP文件路径
    """
    # 确保文件夹路径以斜杠结束
    folder_path = folder_path.rstrip('/') + '/'

    # 创建ZIP文件
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # 遍历文件夹
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # 获取文件的完整路径
                file_path = os.path.join(root, file)
                # 计算在ZIP文件中的相对路径
                relative_path = os.path.relpath(file_path, folder_path)
                # 将文件添加到ZIP文件中
                zipf.write(file_path, relative_path)

# 示例使用
folder_to_zip = '/home/ma-user/work/cylinder_result_pic'  # 要压缩的文件夹路径
output_zip_file = '/home/ma-user/work/cylinder_result_pic.zip'  # 输出的ZIP文件路径
zip_folder(folder_to_zip, output_zip_file)

In [18]:
import moxing as mox
mox.file.copy('/home/ma-user/work/result_pic.zip', 'obs://dirlity/Out/result_pic.zip')

INFO:root:Using MoXing-v2.1.0.5d9c87c8-5d9c87c8
INFO:root:Using OBS-Python-SDK-3.20.9.1
/home/ma-user/anaconda3/envs/PyTorch-1.10.2/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)
